In [1]:
import pandas as pd
from pprint import pprint
import requests
import praw
import time

## Overview
In this notebook I use pushshift api and PRAW to gather close to 20,000 entries from the "critical shower thoughts" and "explain like I'm five" subreddits.

**Pulling from Critical Shower Thoughts**

To get the post from the Critical Shower Thoughts subreddit I reran this code manually until I had enough data. I would like to standardize this code block so it only needs to run once every time this notebook is run but I did not have enough time. In total this collection process took about an hour.

In [24]:
url = 'https://api.pushshift.io/reddit/search/submission'
current_time = 1620842653
posts = []
last_post_time = current_time

In [27]:
fields = ['created_utc', 'selftext']


for i in range(100):
    print(i)
    params = {
    'subreddit': 'C_S_T',
    'size': 100,
    'before': last_post_time,
    'fields' : fields
    }
    
    res = requests.get(url, params)
    
    [posts.append(post) for post in res.json()['data']]
    
    last_post_time = posts[-1]['created_utc']
    
    time.sleep(1.5)
        

0
1


In [206]:
cst_df = pd.DataFrame(posts)

cst_df['target'] = 'cst'

# commented so I don't accidentally overwrite

# cst_df.to_csv('cst_df.csv', index=False)

**Pulling from Explain Like I'm Five**

Here I am scraping only the top comment from posts with more than 20 comments. Posts with more than 10 comments typically have a thoughtfully written out post explaining how something works. Posts with less can have top comments that are short or poorly written. I reran the loop manually until I had enough data.

In [7]:
reddit = praw.Reddit(
    client_id = 'mAzFWiInWHBwSQ',
    client_secret = 'fzQeRPrERWVPJClQ8oU_h2fMsjV6AQ',
    password = 'reddit2357UKL',
    username = 'onwhodoyoudovoodoo',
    user_agent = '/u/onwhodoyoudovoodoo.personal_api_for_learning_python:project_3:v0.0.1'
)

eli5 = reddit.subreddit('explainlikeimfive')

In [226]:
current_time = 1620842653
last_post_time = current_time
eli5_lst = []

In [29]:
url = 'https://api.pushshift.io/reddit/search/submission'
for i in range(100):
    
    # get and filter posts using pushshift
    params = {
    'subreddit': 'explainlikeimfive',
    'size': 100,
    'before': last_post_time,
    }
    
    res = requests.get(url, params)
    data = res.json()
    posts = data['data']
    
    for post in posts:
        
        if post['num_comments'] > 10:
            # get top comment using PRAW
            submission = reddit.submission(id=post['id'])
            # try except because if top comment was deleted
            # it would break code
            try:
                if submission.comments[0].stickied:
                    eli5_lst.append({'created_utc': submission.comments[1].created_utc,
                                     'selftext': submission.comments[1].body,
                                     'target': 'eli5'})
                else:
                    eli5_lst.append({'created_utc': submission.comments[0].created_utc,
                                     'selftext': submission.comments[0].body,
                                     'target': 'eli5'})
            except:
                pass
            
last_post_time = posts[-1]['created_utc']

In [286]:
eli5_df = pd.DataFrame(eli5_lst).drop(columns='id')

# commented so I don't accidentally overwrite

# eli5_df.to_csv('eli5_df.csv', index=False)

## Next Step

Next I perform EDA and feature extraction on these datasets in the data_cleaning notebook.